In [2]:
!pip install mmdet


In [3]:
!pip install mmcv-full


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.9/607.9 kB 11.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for mmcv-full: filename=mmcv_full-1.7.2-cp310-cp310-linux_x86_64.whl size=6107568 sha256=22f36718073bc3f33015a38e42adc938a61180d1f7ea18d9d6dc093194a55efa
  Stored in directory: /root/.cache/pip/wheels/4e/10/0a/78fbabe3d28cc25866432d4ecd6c9b124b8f0a6f546974bb7e
Successfully built mmcv-full


In [ ]:
!pip install mmdet==2.28.0
!pip install mmcv-full==1.7.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: mmdet
    Found existing installation: mmdet 3.3.0
    Uninstalling mmdet-3.3.0:
      Successfully uninstalled mmdet-3.3.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.4/605.4 kB 13.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [637690 lines of output]
      /tmp/pip-install-b0h8pelt/mmcv-full_4d0f2b54f40f4bdc814d80f0877b4fcc/setup.py:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
        from pkg_resources import DistributionNotFound, get_distribution
      [11/17/24 22:06:40] ERROR    listing git files failed - pretending     ]8;id=119543;file:///opt/conda/lib/python3.10/site-packages/setuptools_scm/_file_finders/git.py\git.py]8;;\:]8;id=973962;file:///op

In [ ]:
from mmdet.apis import init_detector, inference_detector, train_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmcv import Config


In [ ]:
# mmdetection/configs/_base_/models/mask_rcnn_r50_fpn.py
_base_ = [
    '../_base_/datasets/coco_detection.py',
    '../_base_/schedules/schedule_1x.py',
    '../_base_/default_runtime.py'
]

model = dict(
    type='CBNet',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch'),
    neck=dict(type='FPN', in_channels=[256, 512, 1024, 2048], out_channels=256, num_outs=5),
    roi_head=dict(
        type='StandardRoIHead',
        bbox_roi_extractor=dict(
            type='SingleRoIExtractor',
            roi_layer=dict(type='RoIAlign', output_size=7, sampling_ratio=2),
            out_channels=256,
            featmap_strides=[4, 8, 16, 32]),
        bbox_head=dict(
            type='Shared4Conv1FCBBoxHead',
            in_channels=256,
            conv_out_channels=256,
            fc_out_channels=1024,
            roi_feat_size=7,
            num_classes=20,  
            bbox_coder=dict(type='DeltaXYWHBBoxCoder', target_means=[0., 0., 0., 0.], target_stds=[0.1, 0.1, 0.2, 0.2]),
            reg_class_agnostic=False,
            loss_cls=dict(type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0),
            loss_bbox=dict(type='L1Loss', loss_weight=1.0)),
    ),
)

dataset_type = 'CocoDataset'
data_root = '/kaggle/input/sat-img-all'

data = dict(
    samples_per_gpu=2,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        ann_file=f'{data_root}/annotations/train.json',
        img_prefix=f'{data_root}/train_images/'),
    val=dict(
        type=dataset_type,
        ann_file=f'{data_root}/annotations/val.json',
        img_prefix=f'{data_root}/val_images/'),
    test=dict(
        type=dataset_type,
        ann_file=f'{data_root}/annotations/test.json',
        img_prefix=f'{data_root}/test_images/'),
)

# Optimizer
optimizer = dict(type='SGD', lr=0.02, momentum=0.9, weight_decay=0.0001)

# Learning rate scheduler
lr_config = dict(
    policy='step',
    step=[8, 11]
)

# Training schedule
total_epochs = 12

# Checkpoint and logging
checkpoint_config = dict(interval=1)
log_config = dict(interval=50)

# Runtime settings
work_dir = './work_dirs/cbnet_mask_rcnn'
load_from = None
resume_from = None
workflow = [('train', 1)]


In [ ]:
from mmdet.apis import train_detector

cfg = Config.fromfile('config_file.py')
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
datasets = [build_dataset(cfg.data.train)]

train_detector(model, datasets, cfg, distributed=False, validate=True)


In [ ]:
from mmdet.apis import init_detector, inference_detector

# Initialize the model
model = init_detector('config_file.py', 'trained_model.pth', device='cuda:0')

# Inference
result = inference_detector(model, '/kaggle/input/sat-img-all/merged_train_images/Abahani-Field_png.rf.3f78f820459a90776d276cd92a05b611.jpg')

# Visualize the results
model.show_result('/kaggle/input/sat-img-all/merged_train_images/Abahani-Field_png.rf.3f78f820459a90776d276cd92a05b611.jpg', result, score_thr=0.5, out_file='result.jpg')
